<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/work_vijitha/Second_try_Feature_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("all_dumimies_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+-----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+---------+-----------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+-----------------

In [4]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:


#####################Create the feature data and the target

#Feature DataFrame is X
X = df.drop(columns=["Id", "SalePrice"])


#Target Dataframe is y
y = df["SalePrice"]


#####################Create train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)



In [7]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [8]:
importances = rf.feature_importances_
importances

array([1.20144471e-02, 2.66270141e-02, 3.01577430e-02, 1.56445897e-02,
       1.18696207e-02, 2.52393683e-02, 2.36159688e-02, 1.59700151e-02,
       2.39938398e-02, 6.26789829e-03, 2.98853145e-02, 2.87334225e-02,
       3.06718451e-02, 1.62204011e-02, 1.65309659e-03, 3.06618113e-02,
       8.29118024e-03, 2.91760888e-03, 6.22936870e-03, 6.89607652e-03,
       1.22761593e-02, 1.45604207e-03, 1.84102305e-02, 1.01551581e-02,
       2.55837194e-02, 7.85040774e-03, 3.00582198e-02, 2.01047255e-02,
       2.22277873e-02, 7.47045598e-03, 2.56784057e-03, 6.29385905e-03,
       4.72597096e-04, 2.36764911e-03, 2.53305668e-02, 1.87636543e-02,
       1.90499702e-03, 1.06461809e-03, 4.39049513e-03, 3.46786905e-03,
       3.92201535e-04, 1.80297869e-03, 1.54355073e-03, 6.73100545e-04,
       7.73960871e-03, 2.37187963e-03, 9.47896371e-04, 3.76762452e-03,
       0.00000000e+00, 2.49778533e-03, 2.19779699e-03, 4.75557902e-04,
       7.85178368e-03, 2.08389187e-03, 4.40576146e-04, 2.42222417e-04,
      

In [9]:


features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)



[(0.03067184507694324, '1stFlrSF'),
 (0.030661811314982232, 'GrLivArea'),
 (0.030157742992857357, 'LotArea'),
 (0.030058219806715736, 'GarageArea'),
 (0.02988531453794777, 'BsmtUnfSF'),
 (0.028733422469003674, 'TotalBsmtSF'),
 (0.026627014082029762, 'LotFrontage'),
 (0.025583719381550338, 'GarageYrBlt'),
 (0.02533056678489419, 'MoSold'),
 (0.025239368282836506, 'YearBuilt'),
 (0.02399383975829967, 'BsmtFinSF1'),
 (0.023615968787044927, 'YearRemodAdd'),
 (0.02222778731027863, 'OpenPorchSF'),
 (0.020104725477762667, 'WoodDeckSF'),
 (0.018763654347624222, 'YrSold'),
 (0.018410230455695117, 'TotRmsAbvGrd'),
 (0.01622040112927497, '2ndFlrSF'),
 (0.015970015071569303, 'MasVnrArea'),
 (0.01564458969957334, 'OverallQual'),
 (0.012276159252435206, 'BedroomAbvGr'),
 (0.01201444710207882, 'MSSubClass'),
 (0.011869620651636485, 'OverallCond'),
 (0.010155158114020211, 'Fireplaces'),
 (0.008291180235079142, 'BsmtFullBath'),
 (0.007851783677985063, 'LotConfig_Inside'),
 (0.007850407737613738, 'Garage